# Imports -> Data Loading -> Handling Class Imbalance :

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import missingno as msno 
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import helpers

In [2]:
data = pd.read_csv("../data/body_level_classification_train.csv")
x = data.drop('Body_Level', axis=1)
y = data['Body_Level']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=31) #31

class_column = 'Body_Level'
class_labels = ['Body Level 1', 'Body Level 2', 'Body Level 3', 'Body Level 4']
class_colors = ['blue','orange', 'green', 'red']
numeric_columns = x_train.select_dtypes(include=[float, int]).columns
categorical_columns = [x for x in x_train.columns if x not in numeric_columns]

print("Training set size:", len(x_train))
print("Testing set size:", len(x_test))


Training set size: 1181
Testing set size: 296


In [3]:
x_train_sampled, y_train_sampled = helpers.class_balance(x_train, y_train, categorical_columns, oversampling_only=True)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6d3853b040>
Traceback (most recent call last):
  File "/home/makrion/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/makrion/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/makrion/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/makrion/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>

# Evaluating Models

### I will go on with those classifiers:
    1. adaboost.
    2. SVM.
    3. random forrest
    4. logistic Regression.

##### adaboost: 


In [4]:
# Create a base classifier
base_classifier = DecisionTreeClassifier(max_depth=5)
# Create an AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(estimator=base_classifier, n_estimators=50, random_state=42)
print("adaboost classifier results:\n")
adaboost_classifier, doc = helpers.test_model(adaboost_classifier, x_train_sampled, y_train_sampled, x_test, y_test)

adaboost classifier results:

     out of sample ( predict(test) ):
         accuracy: 97.63513513513513
         weighted F1 score: 97.63735011416699

     in sample ( predict(train) ):
         accuracy: 100.0
         weighted F1 score: 100.0


##### SVM.

In [5]:
# Create svm classifier
svm_classifier = SVC(kernel='linear', random_state=42)
print("svm classifier results:\n")
svm_classifier, doc = helpers.test_model(svm_classifier, x_train_sampled, y_train_sampled, x_test, y_test)

svm classifier results:

     out of sample ( predict(test) ):
         accuracy: 92.56756756756756
         weighted F1 score: 92.34305620980497

     in sample ( predict(train) ):
         accuracy: 96.36871508379889
         weighted F1 score: 96.3405849945974


##### random forrest:

In [6]:
# Create random forrest classifier
randomforrest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
print("random forrest classifier results:\n")
randomforrest_classifier, doc = helpers.test_model(randomforrest_classifier, x_train_sampled, y_train_sampled, x_test, y_test)

random forrest classifier results:

     out of sample ( predict(test) ):
         accuracy: 96.28378378378379
         weighted F1 score: 96.35971994977031

     in sample ( predict(train) ):
         accuracy: 100.0
         weighted F1 score: 100.0


##### logistic Regression:

In [7]:
# Create a logistic regression classifier
logisticregression_classifier = LogisticRegression(max_iter=10000)
print("logistic regression classifier results:\n")
logisticregression_classifier, doc = helpers.test_model(logisticregression_classifier, x_train_sampled, y_train_sampled, x_test, y_test)

logistic regression classifier results:

     out of sample ( predict(test) ):
         accuracy: 90.54054054054053
         weighted F1 score: 90.31441756165867

     in sample ( predict(train) ):
         accuracy: 91.4804469273743
         weighted F1 score: 91.44529786280899
